In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MA_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1193108802,MA,"417 Liberty St, Springfield, MA",42.114272,-72.585082,2.501380e+14
1,1588717109,MA,"600 Providence Highway, DEDHAM, MA",42.243154,-71.173284,2.502140e+14
2,1636668806,MA,"10 Gove St, East Boston, MA",42.372271,-71.038169,2.502505e+14
3,3105278807,MA,"10 Dearborn Rd, Peabody, MA",42.523734,-70.988951,2.500921e+14
4,3505097107,MA,"1700 IRON HORSE PARK, NORTH BILLERICA, MA",42.581955,-71.266540,2.501732e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1193108802,MA,"417 Liberty St, Springfield, MA",42.114272,-72.585082,2.501380e+14
1,1588717109,MA,"600 Providence Highway, DEDHAM, MA",42.243154,-71.173284,2.502140e+14
2,1636668806,MA,"10 Gove St, East Boston, MA",42.372271,-71.038169,2.502505e+14
3,3105278807,MA,"10 Dearborn Rd, Peabody, MA",42.523734,-70.988951,2.500921e+14
4,3505097107,MA,"1700 IRON HORSE PARK, NORTH BILLERICA, MA",42.581955,-71.266540,2.501732e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
192,1986777209,MA,"171 PARK AVE STE 105, W SPRINGFIELD, MA",40.853719,-73.967067,NaN
443,3179517106,MA,"181 PARK AVE STE 5, WEST SPRINGFIELD, MA",40.804326,-73.939393,NaN
1473,8294267101,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,NaN
1474,9817858407,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,NaN
1974,7792448504,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,NaN
1975,8283097103,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,NaN
3400,9920667106,MA,"85 commercial street, 02155, MA",42.407714,-71.089235,NaN
6219,9178467709,MA,"85 Douglas Pike Back Annex, Smithfield, MA",41.878805,-71.472838,NaN
6399,7483557208,MA,"1191 Chestnut St Suite 2-4, Newton,, MA",39.950560,-75.160020,NaN
8734,4950267102,MA,"PO Box 508, East Bridgewater, MA",42.033434,-70.959210,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('25')] #MA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/25 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

25


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11865/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
192,1986777209,MA,"171 PARK AVE STE 105, W SPRINGFIELD, MA",40.853719,-73.967067,[]
443,3179517106,MA,"181 PARK AVE STE 5, WEST SPRINGFIELD, MA",40.804326,-73.939393,[]
1473,8294267101,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,[250173686001000]
1474,9817858407,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,[250173686001000]
1974,7792448504,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,[250173686001000]
1975,8283097103,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,[250173686001000]
3400,9920667106,MA,"85 commercial street, 02155, MA",42.407714,-71.089235,[250173398021015]
6219,9178467709,MA,"85 Douglas Pike Back Annex, Smithfield, MA",41.878805,-71.472838,[]
6399,7483557208,MA,"1191 Chestnut St Suite 2-4, Newton,, MA",39.950560,-75.160020,[]
8734,4950267102,MA,"PO Box 508, East Bridgewater, MA",42.033434,-70.959210,[250235232011012]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11865/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
192,1986777209,MA,"171 PARK AVE STE 105, W SPRINGFIELD, MA",40.853719,-73.967067,NaN
443,3179517106,MA,"181 PARK AVE STE 5, WEST SPRINGFIELD, MA",40.804326,-73.939393,NaN
1473,8294267101,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,250173686001000
1474,9817858407,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,250173686001000
1974,7792448504,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,250173686001000
1975,8283097103,MA,"191 HIGH ST, WALTHAM, MA",42.365399,-71.229041,250173686001000
3400,9920667106,MA,"85 commercial street, 02155, MA",42.407714,-71.089235,250173398021015
6219,9178467709,MA,"85 Douglas Pike Back Annex, Smithfield, MA",41.878805,-71.472838,NaN
6399,7483557208,MA,"1191 Chestnut St Suite 2-4, Newton,, MA",39.950560,-75.160020,NaN
8734,4950267102,MA,"PO Box 508, East Bridgewater, MA",42.033434,-70.959210,250235232011012


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
192,1986777209,MA,"171 PARK AVE STE 105, W SPRINGFIELD, MA",40.853719,-73.967067,NaN
443,3179517106,MA,"181 PARK AVE STE 5, WEST SPRINGFIELD, MA",40.804326,-73.939393,NaN
6219,9178467709,MA,"85 Douglas Pike Back Annex, Smithfield, MA",41.878805,-71.472838,NaN
6399,7483557208,MA,"1191 Chestnut St Suite 2-4, Newton,, MA",39.950560,-75.160020,NaN
9697,1925188408,MA,"58 Brickett Ln, Haverhill, MA",42.812902,-71.075033,NaN
10753,7215517004,MA,"58 BRICKETT LN, HAVERHILL, MA",42.812902,-71.075033,NaN
13168,9520167110,MA,"2525 Commerce Way, Vista, MA",33.149623,-117.223241,NaN
19117,1174077205,MA,"940 waterman ave, East Providence, MA",41.810657,-71.345278,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
192,1986777209,NaN
443,3179517106,NaN
1473,8294267101,250173686001000
1474,9817858407,250173686001000
1974,7792448504,250173686001000
1975,8283097103,250173686001000
3400,9920667106,250173398021015
6219,9178467709,NaN
6399,7483557208,NaN
8734,4950267102,250235232011012


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,396544,1193108802,MA,"417 Liberty St, Springfield, MA",42.114272,-72.585082,2.501380e+14
1,396545,1588717109,MA,"600 Providence Highway, DEDHAM, MA",42.243154,-71.173284,2.502140e+14
2,396546,1636668806,MA,"10 Gove St, East Boston, MA",42.372271,-71.038169,2.502505e+14
3,396547,3105278807,MA,"10 Dearborn Rd, Peabody, MA",42.523734,-70.988951,2.500921e+14
4,396548,3505097107,MA,"1700 IRON HORSE PARK, NORTH BILLERICA, MA",42.581955,-71.266540,2.501732e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,396544,1193108802,MA,"417 Liberty St, Springfield, MA",42.114272,-72.585082,2.501380e+14,NaN
1,396545,1588717109,MA,"600 Providence Highway, DEDHAM, MA",42.243154,-71.173284,2.502140e+14,NaN
2,396546,1636668806,MA,"10 Gove St, East Boston, MA",42.372271,-71.038169,2.502505e+14,NaN
3,396547,3105278807,MA,"10 Dearborn Rd, Peabody, MA",42.523734,-70.988951,2.500921e+14,NaN
4,396548,3505097107,MA,"1700 IRON HORSE PARK, NORTH BILLERICA, MA",42.581955,-71.266540,2.501732e+14,NaN
...,...,...,...,...,...,...,...,...
27174,423718,9721418410,MA,"942 Beacon St, Newton Center, MA",42.330335,-71.200201,2.501737e+14,NaN
27175,423719,9816118406,MA,"100 Weymouth St, Rockland, MA",42.157247,-70.908888,2.502350e+14,NaN
27176,423720,9869577804,MA,"260 MAIN ST, MARSHFIELD, MA",42.102761,-70.719124,2.502351e+14,NaN
27177,423721,9919968406,MA,"102 South St Apt 3, Boston, MA",42.351130,-71.057216,2.502507e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,396544,1193108802,MA,"417 Liberty St, Springfield, MA",42.114272,-72.585082,2.501380e+14,NaN,250138009003008.0
1,396545,1588717109,MA,"600 Providence Highway, DEDHAM, MA",42.243154,-71.173284,2.502140e+14,NaN,250214024004002.0
2,396546,1636668806,MA,"10 Gove St, East Boston, MA",42.372271,-71.038169,2.502505e+14,NaN,250250506001005.0
3,396547,3105278807,MA,"10 Dearborn Rd, Peabody, MA",42.523734,-70.988951,2.500921e+14,NaN,250092103021003.0
4,396548,3505097107,MA,"1700 IRON HORSE PARK, NORTH BILLERICA, MA",42.581955,-71.266540,2.501732e+14,NaN,250173165003016.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1193108802,MA,"417 Liberty St, Springfield, MA",42.114272,-72.585082,250138009003008.0
1,1588717109,MA,"600 Providence Highway, DEDHAM, MA",42.243154,-71.173284,250214024004002.0
2,1636668806,MA,"10 Gove St, East Boston, MA",42.372271,-71.038169,250250506001005.0
3,3105278807,MA,"10 Dearborn Rd, Peabody, MA",42.523734,-70.988951,250092103021003.0
4,3505097107,MA,"1700 IRON HORSE PARK, NORTH BILLERICA, MA",42.581955,-71.266540,250173165003016.0
...,...,...,...,...,...,...
27174,9721418410,MA,"942 Beacon St, Newton Center, MA",42.330335,-71.200201,250173738004004.0
27175,9816118406,MA,"100 Weymouth St, Rockland, MA",42.157247,-70.908888,250235022004000.0
27176,9869577804,MA,"260 MAIN ST, MARSHFIELD, MA",42.102761,-70.719124,250235061024001.0
27177,9919968406,MA,"102 South St Apt 3, Boston, MA",42.351130,-71.057216,250250701041015.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
192,1986777209,MA,"171 PARK AVE STE 105, W SPRINGFIELD, MA",40.853719,-73.967067,NaN
443,3179517106,MA,"181 PARK AVE STE 5, WEST SPRINGFIELD, MA",40.804326,-73.939393,NaN
6219,9178467709,MA,"85 Douglas Pike Back Annex, Smithfield, MA",41.878805,-71.472838,NaN
6399,7483557208,MA,"1191 Chestnut St Suite 2-4, Newton,, MA",39.950560,-75.160020,NaN
9697,1925188408,MA,"58 Brickett Ln, Haverhill, MA",42.812902,-71.075033,NaN
10753,7215517004,MA,"58 BRICKETT LN, HAVERHILL, MA",42.812902,-71.075033,NaN
13168,9520167110,MA,"2525 Commerce Way, Vista, MA",33.149623,-117.223241,NaN
19117,1174077205,MA,"940 waterman ave, East Providence, MA",41.810657,-71.345278,NaN


Convert to File

In [19]:
fips_MA = fips_merge_drop

In [20]:
fips_MA.to_csv("../../../data/state_data/geo/geo_fips/23/MA_fips_scraped.csv") 